In [2]:
import pandas as pd

model_names = ["gpt-3.5-turbo", "gpt-4-turbo", "gpt-4o"]

for model_name in model_names:
    df = pd.read_csv(f"../../../results/{model_name}_jailbreak_responses_labeled_GeminiEval.csv", header=0)
    zero_count = (df['eval_label'] == 0).sum()   
    total_count = len(df)                        
    ratio = zero_count / total_count
    
    print(f"model: {model_name}")
    print(f"detected: {zero_count}, total: {total_count}, percentage: {ratio:.4f}\n")

model: gpt-3.5-turbo
detected: 908, total: 1405, percentage: 0.6463

model: gpt-4-turbo
detected: 942, total: 1405, percentage: 0.6705

model: gpt-4o
detected: 1175, total: 1405, percentage: 0.8363

